# Accessing SSA Model Data

API version: 1.0

Authors: Craig Pellegrino (NASA GSFC)

## Introduction

This notebook will demonstrate how to access Science Situational Awareness (SSA) model data from the ACROSS core-server using the client library. SSA model data includes `Observatory`/`Telescope`/`Instrument` objects, `Filters`, and `Observations` and `Schedules`. Each of these objects has an API class accessed through the `Client` that can be used to perform `GET` requests to the core-server, retrieving and filtering data based on the user's input parameters.

## 1. Setup and Instantiating the `Client`

The first step to accessing the core-server is to instantiate the `Client` class. Users can pass credentials to the `Client` to perform access-limited tasks, such as `POSTing` schedule data to the server. However all SSA-related `GET` endpoints are available without authentication, so for our purposes here we won't have to worry about credentials.

Let's import the necessary dependencies and instantiate the `Client` object:

In [1]:
from across.client import Client

client = Client()

That's it! `client` can now access the SSA API endpoints to retrieve data. Let's show a few examples of how to do that, including filtering by some common parameters.

## 2. Accessing Observatory/Telescope/Instrument/Filter Data

### 2.1. Observatories

In the ACROSS system, observational resources are captured as `Observatory`, `Telescope`, and `Instrument` objects. The hierarchy is in that order--an `Observatory` can have many `Telescopes`, which in turn can have many `Instruments`. `Instruments` can also have many `Filters`, which describe the wavelength ranges that `Instrument` can observe in a given exposure.

At the top level, an `Observatory` contains data such as a `name`, `type` (`SPACE_BASED` or `GROUND_BASED`), and a collection of telescopes. We can filter and search on any one of those parameters using the client's `observatory.get_many` method, which returns a list of `Observatory` objects that match the input parameters:

In [2]:
# GET an observatory by name
observatories = client.observatory.get_many(name="JWST")
observatory = observatories[0]
observatory.name

'James Webb Space Telescope'

In [3]:
# GET a list of observatories by type
space_based_observatories = client.observatory.get_many(type="SPACE_BASED")
for obs in space_based_observatories:
    print(obs.name)

Imaging X-Ray Polarimetry Explorer
Fermi Gamma-ray Space Telescope
Transiting Exoplanet Survey Satellite
Nuclear Spectroscopic Telescope Array
Neutron Star Interior Composition Explorer
Chandra X-ray Observatory
Neil Gehrels Swift Observatory
Hubble Space Telescope
James Webb Space Telescope
X-ray Multi-Mirror Mission


In [4]:
# GET an observatory by its telescope name
observatories = client.observatory.get_many(telescope_name="UVOT")
observatories[0].name

'Neil Gehrels Swift Observatory'

We can also use the `get` method to look up an observatory by its known `UUID`:

In [5]:
jwst = client.observatory.get(id=observatory.id)
jwst.name

'James Webb Space Telescope'

### 2.2. Telescopes

`Telescopes` function similarly to `Observatories` in that they can be filtered by `name`, `id`, or `Instrument` parameters using the `get_many` method:

In [6]:
# GET a telescope by name
telescopes = client.telescope.get_many(name="UVOT")
telescope = telescopes[0]
telescope.name

'UV/Optical Telescope'

In [7]:
# GET a telescope by its instrument name
telescopes = client.telescope.get_many(instrument_name="NIRCam")
telescopes[0].name

'James Webb Space Telescope'

And there's a `get` method to look up a `Telescope` by `UUID`:

In [8]:
uvot = client.telescope.get(id=telescope.id)
uvot.name

'UV/Optical Telescope'

### 2.3. Instruments

Finally, `Instrument` objects have analogous filtering parameters to `Observatory` and `Telescope` objects with similar `get` and `get_many` methods:

In [9]:
# GET an instrument by name
instruments = client.instrument.get_many(name="NIRCAM")
instrument = instruments[0]
instrument.name

'Near-Infrared Camera'

In [10]:
# GET an instrument by id
nircam = client.instrument.get(id=instrument.id)
nircam.name

'Near-Infrared Camera'

### 2.4. Filters

`Instruments` can have many `Filters`, which describes the wavelength ranges an `Instrument` can cover in a single observation.

In [11]:
# GET filters for an instrument
filters = client.filter.get_many(instrument_name="UVOT")
filt = filters[0]
print(f"Name: {filt.name}")
print(f"Min wavelength: {filt.min_wavelength:.1f}")
print(f"Max wavelength: {filt.max_wavelength:.1f}")

Name: Swift UVOT UVW2
Min wavelength: 1600.0
Max wavelength: 2250.0


Suppose you want to observe over some wavelength range using a given instrument. You can use the client to find all the filters that match these criteria: 

In [12]:
# GET filters by wavelength region
filters = client.filter.get_many(instrument_name="UVOT", covers_wavelength=2300)  # Assumes Angstroms
print([filt.name for filt in filters])

['Swift UVOT UVM2', 'Swift UVOT UVW1', 'Swift UVOT White']


## 3. Accessing Schedule and Observation Data

### 3.1. Schedules

In the ACROSS system, `Schedules` aggregate observation metadata for a given telescope. They are broadly grouped by status (`planned`, `scheduled`, and `as-flown`) and by fidelity (`low` and `high`). `Schedules` can be retrieved and filtered using these parameters as well as others, including date range, observatory/telescope name and ID, and external ID.

`Schedule` objects are optionally paginated--users can supply a `page` (e.g., 1) and `page_limit` (e.g. 10 per page) to limit the number of returned objects.

To retrieve all schedules matching the input parameters, use the `get_many` method:

In [13]:
# GET a paginated list of schedules:
schedules = client.schedule.get_many(page=1, page_limit=10, fidelity="low", status="planned")
print(f"Page: {schedules.page}")
print(f"Number of schedules returned: {len(schedules.items)}")
print(f"Total number of schedules: {schedules.model_dump()['total_number']}")

Page: 1
Number of schedules returned: 10
Total number of schedules: 49


Each of these returned items is a schedule for a given telescope. 

We can use the `model_dump_json` Pydantic schema method to display the schedule data in a nicer format:

In [14]:
print(schedules.items[0].model_dump_json(indent=4))

{
    "telescope_id": "f2ae30ec-cd64-41b1-a951-4da29aa9f4ab",
    "name": "XMM_Newton_planned_2025-10-21_2025-10-31",
    "date_range": {
        "begin": "2025-10-21T12:13:52",
        "end": "2025-10-31T19:43:14"
    },
    "status": "planned",
    "external_id": null,
    "fidelity": "low",
    "id": "b7b0d25c-bc58-48fd-b173-968ee8f2e911",
    "observations": [],
    "observation_count": 190,
    "created_on": "2025-10-21T09:00:21.754546",
    "created_by_id": "e5aa65a1-b231-46e2-aded-381082917a23",
    "checksum": "ec4601f7214b4cde3ed0196177a83ce3749ddca5ea362569c14a8eb95b05e65851d205d54331935f1a4cac30b9432677bd0cfcc6cd995e16cf6d68ca434896e5"
}


If you only want to know about the most up-to-date schedules in the ACROSS system, there's also a `get_history` endpoint, which returns only the most-recently submitted schedule per telescope in a specified date-range: 

In [15]:
# GET the most recent schedules for the input parameters
recent_schedules = client.schedule.get_history(status="planned")
print(f"Number of recent schedules: {recent_schedules.model_dump()['total_number']}")

Number of recent schedules: 51


And finally, there's a `get` endpoint to look up a schedule by its known UUID:

In [16]:
# GET a schedule by UUID
schedule = client.schedule.get(id=recent_schedules.items[0].id)
print(schedule.model_dump_json(indent=4))

{
    "telescope_id": "f2ae30ec-cd64-41b1-a951-4da29aa9f4ab",
    "name": "XMM_Newton_planned_2025-10-21_2025-10-31",
    "date_range": {
        "begin": "2025-10-21T12:13:52",
        "end": "2025-10-31T19:43:14"
    },
    "status": "planned",
    "external_id": null,
    "fidelity": "low",
    "id": "b7b0d25c-bc58-48fd-b173-968ee8f2e911",
    "observations": [],
    "observation_count": 190,
    "created_on": "2025-10-21T09:00:21.754546",
    "created_by_id": "e5aa65a1-b231-46e2-aded-381082917a23",
    "checksum": "ec4601f7214b4cde3ed0196177a83ce3749ddca5ea362569c14a8eb95b05e65851d205d54331935f1a4cac30b9432677bd0cfcc6cd995e16cf6d68ca434896e5"
}


### 3.2. Observations

`Observation` objects contain the specific information about a telescope's exposure at a given time, such as where it was observing and with what instrument and filter. They are grouped into `Schedules` and can be accessed through the returned schedule objects described above (via `schedule.observations`), or through one of two endpoints described here.

Like `Schedule` objects, `Observations` can be optionally paginated.

To retrieve many `Observations`, use the `get_many` method with any number of optional input parameters. Here we will demonstrate the most common ones:

In [17]:
# GET observations by date range
from datetime import datetime

observations = client.observation.get_many(
    page=1, page_limit=10, date_range_begin=datetime(2025, 10, 20), date_range_end=datetime(2025, 10, 21)
)
print(observations.items[0].model_dump_json(indent=4))

{
    "instrument_id": "1c074192-f6d5-465d-844e-85a0010b2d87",
    "object_name": "GRB 251011B",
    "pointing_position": {
        "ra": 13.22209,
        "dec": -62.00911
    },
    "date_range": {
        "begin": "2025-10-20T23:40:00",
        "end": "2025-10-21T00:00:00"
    },
    "external_observation_id": "01403191007",
    "type": "imaging",
    "status": "planned",
    "pointing_angle": 193.682286530267,
    "exposure_time": 1200.0,
    "reason": null,
    "description": null,
    "proposal_reference": null,
    "object_position": {
        "ra": 13.22209,
        "dec": -62.00911
    },
    "depth": null,
    "bandpass": {
        "anyof_schema_1_validator": null,
        "anyof_schema_2_validator": null,
        "anyof_schema_3_validator": null,
        "actual_instance": {
            "filter_name": "Swift UVOT u",
            "min": 3079.9999999999995,
            "max": 3849.9999999999995,
            "type": "WAVELENGTH",
            "central_wavelength": 3464.999999999

In [18]:
# GET observations by cone search
observations = client.observation.get_many(cone_search_ra=120.0, cone_search_dec=-50.0, cone_search_radius=1)
print(f"Number of observations returned: {observations.model_dump()['total_number']}")

Number of observations returned: 4


In [19]:
# GET observations by observatory ID
observatory = client.observatory.get_many(name="HST")[0]
observations = client.observation.get_many(observatory_ids=[observatory.id])
print(f"Number of observations returned: {observations.model_dump()['total_number']}")
print("The same process can be used to get observations by telescope, instrument, and schedule ID")

Number of observations returned: 854
The same process can be used to get observations by telescope, instrument, and schedule ID


In [20]:
# GET observations by bandpass information
observations = client.observation.get_many(bandpass_min=2000.0, bandpass_max=3000.0, bandpass_type="angstrom")
print(f"Number of observations returned: {observations.model_dump()['total_number']}")

Number of observations returned: 1200


In [21]:
# GET observations by status
observations = client.observation.get_many(page=1, page_limit=10, status="planned")
print(f"Number of observations returned: {observations.model_dump()['total_number']}")

Number of observations returned: 34034


And as always there's a `get` method that behaves the same to the others demonstrated so far.